**链接到自己的谷歌云盘**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

**1.安装插件**

```
!pip install openai requests
```

In [34]:
!pip install openai requests

**2.引入openai环境**

注意此处需要您自己的api_key来测试

In [14]:
import openai
import os
api_key = 'sk-'
openai.api_key  = api_key

**3.前期准备：将txt文件修改为我们标准化jsonl格式**

In [10]:
import json

# 打开文件
with open('/content/drive/MyDrive/openai/data/raw/training_data.txt', 'r', encoding='utf-8') as file:
    content = file.read()

# 以三个连续的换行符分割成段落
paragraphs_level1 = content.split('\n\n\n')

# 构建消息列表
messages = []

for i, paragraph_level1 in enumerate(paragraphs_level1, start=1):
    # 在每个段落内再以两个连续的换行符分割内容
    paragraphs_level2 = paragraph_level1.split('\n\n')

    # 初始化消息对象
    message = {"messages": []}

    # 添加系统消息
    message["messages"].append({"role": "system", "content": "你是一名翻译"})

    # 添加第二级段落内容到不同角色的消息
    for j, paragraph_level2 in enumerate(paragraphs_level2, start=1):
        role = "user" if j % 2 == 1 else "assistant"
        message["messages"].append({"role": role, "content": paragraph_level2})

    # 将消息对象添加到消息列表
    messages.append(message)

# 将消息列表写入到 chat_data.jsonl 文件
with open('/content/drive/MyDrive/openai/data/processed/chat_data.jsonl', 'w', encoding='utf-8') as jsonl_file:
    for message in messages:
        jsonl_file.write(json.dumps(message, ensure_ascii=False) + '\n')


4.上传可用于微调功能的文档

In [ ]:
import requests
import openai

url = "https://api.openai.com/v1/files"
headers = {
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "purpose": "fine-tune",
}
files = {
    "file": open("/content/drive/MyDrive/openai/data/processed/chat_data.jsonl", "rb")
}

response = requests.post(url, headers=headers, data=payload, files=files)
response_text = response.text
print(response_text)
response_json = json.loads(response_text)  # 将响应文本转换为 JSON 字典
file_id = response_json["id"]
print(file_id)


5.创建fine-tuned模型,模型结果可以在下面的接口或者邮箱收到

In [ ]:
import requests

url = "https://api.openai.com/v1/fine_tuning/jobs"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}
data = {
    "training_file": f"{file_id}",
    "model": "gpt-3.5-turbo-0613"
}

response = requests.post(url, headers=headers, json=data)
print(response.text)
response_json = json.loads(response.text)  # 将响应文本转换为 JSON 字典
fine_tuning_job_id = response_json["id"]
print(fine_tuning_job_id)

6.查看微调的模型，这一块需要刷新

如果status是running的话就说明该模型还在运行，此时fine_tuned_model里面还没有值

如果status是succeeded说明运行完，此时fine_tuned_model里面的模型可以使用了

In [ ]:
import requests

url = f"https://api.openai.com/v1/fine_tuning/jobs/{fine_tuning_job_id}"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

response = requests.get(url, headers=headers)
print(response.text)
response_json = json.loads(response.text)  # 将响应文本转换为 JSON 字典
fine_tuned_model = response_json["fine_tuned_model"]
print(fine_tuned_model)

7.检测模型结果

In [ ]:
response = openai.ChatCompletion.create(
    model=f"{fine_tuned_model}",
    messages=[
        {"role": "system", "content": "这里写入需要检测的system内容"},
        {"role": "user", "content": "这里输入需要检测的问题内容"}

    ]
)
print(response.choices[0].message.content)